In [ ]:
# код работает с примерами на сложение
from itertools import permutations
import time


def word_val(letter_values, power_of_ten):
    val = 0
    for i in range(len(letter_values)):
        val += letter_values[i] * power_of_ten[i]
    return val


if __name__ == "__main__":
    x = input("Первое слово: ").upper()
    y = input("Второе слово: ").upper()
    z = input("Сумма двух слов: ").upper()

    t0 = time.time()

    # Рассчитаем силу 10, которые соответствуют каждой букве.
    # чтобы сэкономить время при каждом вызове функции word_val.
    xProd = [10 ** i for i in range(len(x) - 1, -1, -1)]
    yProd = [10 ** i for i in range(len(y) - 1, -1, -1)]
    zProd = [10 ** i for i in range(len(z) - 1, -1, -1)]

    # Найдем все разные буквы, которые существуют в крипторифметическом уравнении.
    letters = []
    for i in x + y + z:
        if i not in letters:
            letters.append(i)

    # Если len(letters) > 10 решений не существует, потому что каждая буква должна иметь уникальное значение 0-9.
    num_of_letters = len(letters)
    if num_of_letters > 10:
        print("Нет Решения, слишком много разных букв.")
        exit()

    permList = (list(permutations([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], num_of_letters)))

    firstLetterIndex = [letters.index(x[0]), letters.index(y[0]), letters.index(z[0])]

    # Если да, то первая буква результирующего слова - 1.
    carryFlag = False
    if len(z) > len(x) and len(z) > len(y):
        carryFlag = True

    # printFlag: чтобы проверить, найдено ли решение или нет.
    # Добавление в список без необходимости замедляет работу программы.
    printFlag = False
    print(f"{x} + {y} = {z}")
    for perm in permList:

        if carryFlag and perm[firstLetterIndex[-1]] != 1:
            continue
        for let_index in firstLetterIndex[:-1]:
            if perm[let_index] != 0:
                continue

        x_val = [0] * len(x)
        y_val = [0] * len(y)
        z_Val = [0] * len(z)

        for i in range(num_of_letters):
            indices = [k for k, j in enumerate(x) if j == letters[i]]
            for j in indices:
                x_val[j] = perm[i]
            indices = [k for k, j in enumerate(y) if j == letters[i]]
            for j in indices:
                y_val[j] = perm[i]
            indices = [k for k, j in enumerate(z) if j == letters[i]]
            for j in indices:
                z_Val[j] = perm[i]

        xEquiv = word_val(x_val, xProd)
        yEquiv = word_val(y_val, yProd)
        zEquiv = word_val(z_Val, zProd)
        if xEquiv + yEquiv == zEquiv:
            xSol = ''.join(str(e) for e in x_val)
            ySol = ''.join(str(e) for e in y_val)
            zSol = ''.join(str(e) for e in z_Val)
            print(f"{xSol} + {ySol} = {zSol}")
            printFlag = True

    if not printFlag:
        print("Решений не найдено.")
        exit()

    t1 = time.time()
    print(f"\nВыполнено за: {round(t1 - t0)}s")


In [ ]:
# brute force код решит любую крипторифму, но не быстро
# работает корректно при синтаксе:' слово знак слово == слово'


def solve(cryptarithm: str):
    starting_letter = set()
    not_starting_letters = set()
    in_word = False
    for c in cryptarithm:
        if str.isalpha(c):
            if not in_word:
                starting_letter.add(c)
                in_word = True
            else:
                not_starting_letters.add(c)
        else:
            in_word = False
    not_starting_letters -= starting_letter
    all_letters = (''.join(not_starting_letters) + ''.join(starting_letter))
    if len(all_letters) > 10:
        print('больше 10 букв, это не криптарифм.')
    for possible_sol in generate_possible_sol([str(i) for i in range(10)], len(starting_letter),
                                              len(not_starting_letters)):
        equation = cryptarithm.translate(str.maketrans(all_letters, possible_sol))
        try:
            if eval(equation):
                print(equation)
        except ArithmeticError:
            pass
        except:
            print('Неправильное выражение, это не криптарифм.')


def load(m, start, size, length):
    if length == 0:
        yield m
        if size > 1:
            if size % 2 or size == 2:
                m[start], m[start + size - 1] = m[start + size - 1], m[start]
            elif size == 4:
                m[start: start + 3], m[start + 3] = m[start + 1: start + 4], m[start]
            else:
                m[start: start + 2], m[start + 2: start + size - 2], m[start + size - 2], \
                m[start + size - 1] = m[start + size - 3: start + size - 1], \
                                      m[start + 1: start + size - 3], m[start + size - 1], m[start]
    else:
        size -= 1
        length -= 1
        for i in range(size):
            for m in load(m, start, size, length):
                yield m
            if size % 2:
                m[start + i], m[start + size] = m[start + size], m[start + i]
            else:
                m[start], m[start + size] = m[start + size], m[start]
        for m in load(m, start, size, length):
            yield m


def generate_possible_sol(m, size1, size2):
    adjusted_size2 = size2
    if size2 == 10 - size1:
        adjusted_size2 -= 1
    adjusted_size1 = size1
    if size1 == 9:
        adjusted_size1 -= 1
    start = 10 - size1 - size2
    for m in load(m, 1, 9, adjusted_size1):
        mm = list(m)
        for mm in load(mm, 0, 10 - size1, adjusted_size2):
            yield ''.join(mm[start:])


if __name__ == "__main__":
    cryptarithm = 'ДВА * ДВА == ЧЕТЫРЕ'
    print(cryptarithm)
    solve(cryptarithm)


полезные ссылки:
---
- https://habr.com/ru/post/189050/
- https://ridero.ru/books/chislovye_rebusy/freeText
- http://rebus1.com/index.php?item=cryptarithm